## Week 4, Lab 2: Predicting Chronic Kidney Disease in Patients
> Author: Matt Brems

We can sketch out the data science process as follows:
1. Define the problem.
2. Obtain the data.
3. Explore the data.
4. Model the data.
5. Evaluate the model.
6. Answer the problem.

In this lab, we're going to focus on steps exploring data, building models and evaluating the models we build.

There are three links you may find important:
- [A set of chronic kidney disease (CKD) data and other biological factors](./chronic_kidney_disease_full.csv).
- [The CKD data dictionary](./chronic_kidney_disease_header.txt).
- [An article comparing the use of k-nearest neighbors and support vector machines on predicting CKD](./chronic_kidney_disease.pdf).

## Step 1: Define the problem.

Suppose you're working for Mayo Clinic, widely recognized to be the top hospital in the United States. In your work, you've overheard nurses and doctors discuss test results, then arrive at a conclusion as to whether or not someone has developed a particular disease or condition. For example, you might overhear something like:

> **Nurse**: Male 57 year-old patient presents with severe chest pain. FDP _(short for fibrin degradation product)_ was elevated at 13. We did an echo _(echocardiogram)_ and it was inconclusive.

> **Doctor**: What was his interarm BP? _(blood pressure)_

> **Nurse**: Systolic was 140 on the right; 110 on the left.

> **Doctor**: Dammit, it's an aortic dissection! Get to the OR _(operating room)_ now!

> _(intense music playing)_

In this fictitious but [Shonda Rhimes-esque](https://en.wikipedia.org/wiki/Shonda_Rhimes#Grey's_Anatomy,_Private_Practice,_Scandal_and_other_projects_with_ABC) scenario, you might imagine the doctor going through a series of steps like a [flowchart](https://en.wikipedia.org/wiki/Flowchart), or a series of if-this-then-that steps to diagnose a patient. The first steps made the doctor ask what the interarm blood pressure was. Because interarm blood pressure took on the values it took on, the doctor diagnosed the patient with an aortic dissection.

Your goal, as a research biostatistical data scientist at the nation's top hospital, is to develop a medical test that can improve upon our current diagnosis system for [chronic kidney disease (CKD)](https://www.mayoclinic.org/diseases-conditions/chronic-kidney-disease/symptoms-causes/syc-20354521).

**Real-world problem**: Develop a medical diagnosis test that is better than our current diagnosis system for CKD.

**Data science problem**: Develop a medical diagnosis test that reduces both the number of false positives and the number of false negatives.

In [1]:
# Imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import \
    (confusion_matrix, 
     plot_confusion_matrix, 
     accuracy_score, 
     plot_roc_curve, 
     roc_auc_score, 
     recall_score, 
     precision_score, 
     f1_score)

---

## Step 2: Obtain the data.

### 1. Read in the data.

In [2]:
ckd_old = pd.read_csv('chronic_kidney_disease_full.csv')
ckd = ckd_old

In [3]:
ckd.shape

(400, 25)

In [4]:
ckd.dtypes

age      float64
bp       float64
sg       float64
al       float64
su       float64
rbc       object
pc        object
pcc       object
ba        object
bgr      float64
bu       float64
sc       float64
sod      float64
pot      float64
hemo     float64
pcv      float64
wbcc     float64
rbcc     float64
htn       object
dm        object
cad       object
appet     object
pe        object
ane       object
class     object
dtype: object

### 2. Check out the data dictionary. What are a few features or relationships you might be interested in checking out?

**Answer:**

1. Relationship between the different variables with ckd (class column which will be our y variable)

2. Whether or not some variables have outliers.

|Column Name|Type|Description|Values|
|-----------|----|-----------|------|
|age | (numerical) | age in years| 2 - 90|
|bp  | (numerical) | blood pressure| bp in mm/Hg |
|sg	 | (nominal)   | specific gravity | 1.005,1.010,1.015,1.020,1.025|
|al  | (nominal)   |albumin | 0, 1, 2, 3, 4, 5|
|su  | (nominal)   |sugar | 0, 1, 2, 3, 4, 5|
|rbc | (nominal)   | Red Blood Cells | normal, abnormal|
|pc	 | (nominal)   | Pus Cell| normal, abnormal|
|pcc | (nominal)   | Pus Cell Clumps| normal, abnormal|
|ba	 | (nominal)   | bacteria| present, notpresent
|bgr | (nominal)   | blood glucose random | bgr in mgs/dl
|bu	 | (numerical) | blood urea | bu in mgs/dl
|sc	 | (numerical) | serum creatinine | sc in mgs/dl
|sod | (numerical) | sodium |sod in mEq/L
|pot | (numerical) | potassium | sod in mEq/L
|hemo| (numerical) | hemoglobin | hemo in gms
|pcv | (numerical) | packed cell volume||
|wc	 | (numerical) | white blood cell count| wc in cells/cumm|
|rc	 | (numerical) | red blood cell count| rc in millions/cmm
|htn | (nominal) | hypertension | yes, no|
|dm	 | (nominal) | diabetes mellitus| yes, no|
|cad | (nominal) | coronary artery disease| yes, no|
|appet| (nominal)| appetite | good, poor|
|pe	 | (nominal) | pedal edema | yes, no|
|ane | (nominal) | anemia | yes, no|
|class| (nominal)| class | ckd, notckd


---

## Step 3: Explore the data.

### 3. How much of the data is missing from each column?

In [5]:
ckd.isnull().sum()

age        9
bp        12
sg        47
al        46
su        49
rbc      152
pc        65
pcc        4
ba         4
bgr       44
bu        19
sc        17
sod       87
pot       88
hemo      52
pcv       71
wbcc     106
rbcc     131
htn        2
dm         2
cad        2
appet      1
pe         1
ane        1
class      0
dtype: int64

### 4. Suppose that I dropped every row that contained at least one missing value. (In the context of analysis with missing data, we call this a "complete case analysis," because we keep only the complete cases!) How many rows would remain in our dataframe? What are at least two downsides to doing this?

> There's a good visual on slide 15 of [this deck](https://liberalarts.utexas.edu/prc/_files/cs/Missing-Data.pdf) that shows what a complete case analysis looks like if you're interested.

**Answer:**

If we delete anything which has an empty value, we dropped 242 rows and are left with only 39.5% of the original data.

1. This will make our model less accurate due to less observations.
2. The proportions of 'class': whether or not they have ckd, has changed drastically due to the number of drops.

In [6]:
ckd.dropna(how='any')['age'].count()

158

In [7]:
ckd['class'].value_counts(normalize=True)

ckd       0.625
notckd    0.375
Name: class, dtype: float64

In [8]:
ckd.dropna(how='any')['class'].value_counts(normalize=True)

notckd    0.727848
ckd       0.272152
Name: class, dtype: float64

### 5. Thinking critically about how our data were gathered, it's likely that these records were gathered by doctors and nurses. Brainstorm three potential areas (in addition to the missing data we've already discussed) where this data might be inaccurate or imprecise.

**Answer:** 

1. Some data are subjective. For example, a person's appetite may be 'good' or 'poor' based on subjective feelings. 
2. Some doctors may check certain things and not others. As such, some data are missing because the doctor does not think it is a cause of concern.
3. Blood glucose (random) may be different since some patients might have just finished eating and others may have extended fasting periods.

---

## Step 4: Model the data.

### 6. Suppose that I want to construct a model where no person who has CKD will ever be told that they do not have CKD. What (very simple, no machine learning needed) model can I create that will never tell a person with CKD that they do not have CKD?

> Hint: Don't think about `statsmodels` or `scikit-learn` here.

**Answer:** This means that there will be no negatives. So only true and false positives, which means that we tell everyone that they have CKD.

### 7. In problem 6, what common classification metric did we optimize for? Did we minimize false positives or negatives?

**Answer:** We minimize false negatives by optimizing Sensitivity (True Positive / All Positive).

### 8. Thinking ethically, what is at least one disadvantage to the model you described in problem 6?

**Answer:** Misinformation and wasting patient's money will cause complaints

### 9. Suppose that I want to construct a model where a person who does not have CKD will ever be told that they do have CKD. What (very simple, no machine learning needed) model can I create that will accomplish this?

**Answer:** This means that there will be no positives. So all values are negative and we tell everyone that they do not have CKD.

### 10. In problem 9, what common classification metric did we optimize for? Did we minimize false positives or negatives?

**Answer:** We optimize for Specificity. This minimizes false positives.

### 11. Thinking ethically, what is at least one disadvantage to the model you described in problem 9?

**Answer:** Not telling people who have a disease, and they will die from it, and we get sued.

### 12. Construct a logistic regression model in `sklearn` predicting class from the other variables. You may scale, select/drop, and engineer features as you wish - build a good model! Make sure, however, that you include at least one categorical/dummy feature and at least one quantitative feature.

> Hint: Remember to do a train/test split!

### Making dummies for string columns

In [9]:
categorical_columns = ['rbc', 'pc', 'pcc', 'ba', 'htn', 'dm', 'cad', 'appet', 'pe', 'ane']
pd.get_dummies(data=ckd, columns=categorical_columns)

,age,bp,sg,al,su,bgr,bu,sc,sod,pot,...,dm_no,dm_yes,cad_no,cad_yes,appet_good,appet_poor,pe_no,pe_yes,ane_no,ane_yes
0,48.0,80.0,1.020,1.0,0.0,121.0,36.0,1.2,NaN,NaN,...,0,1,1,0,1,0,1,0,1,0
1,7.0,50.0,1.020,4.0,0.0,NaN,18.0,0.8,NaN,NaN,...,1,0,1,0,1,0,1,0,1,0
2,62.0,80.0,1.010,2.0,3.0,423.0,53.0,1.8,NaN,NaN,...,0,1,1,0,0,1,1,0,0,1
3,48.0,70.0,1.005,4.0,0.0,117.0,56.0,3.8,111.0,2.5,...,1,0,1,0,0,1,0,1,0,1
4,51.0,80.0,1.010,2.0,0.0,106.0,26.0,1.4,NaN,NaN,...,1,0,1,0,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,55.0,80.0,1.020,0.0,0.0,140.0,49.0,0.5,150.0,4.9,...,1,0,1,0,1,0,1,0,1,0
396,42.0,70.0,1.025,0.0,0.0,75.0,31.0,1.2,141.0,3.5,...,1,0,1,0,1,0,1,0,1,0
397,12.0,80.0,1.020,0.0,0.0,100.0,26.0,0.6,137.0,4.4,...,1,0,1,0,1,0,1,0,1,0
398,17.0,60.0,1.025,0.0,0.0,114.0,50.0,1.0,135.0,4.9,...,1,0,1,0,1,0,1,0,1,0


### Using .map to change Class into ckd_yes

In [10]:
ckd['class'] = ckd['class'].map(lambda x: 1 if x == 'ckd' else 0)
ckd.rename(columns={'class': 'ckd_yes'}, inplace=True)
ckd.reset_index(drop=True)
ckd.head(2)

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wbcc,rbcc,htn,dm,cad,appet,pe,ane,ckd_yes
0,48.0,80.0,1.02,1.0,0.0,NaN,normal,notpresent,notpresent,121.0,...,44.0,7800.0,5.2,yes,yes,no,good,no,no,1
1,7.0,50.0,1.02,4.0,0.0,NaN,normal,notpresent,notpresent,NaN,...,38.0,6000.0,NaN,no,no,no,good,no,no,1


In [11]:
ckd.isnull().sum()

age          9
bp          12
sg          47
al          46
su          49
rbc        152
pc          65
pcc          4
ba           4
bgr         44
bu          19
sc          17
sod         87
pot         88
hemo        52
pcv         71
wbcc       106
rbcc       131
htn          2
dm           2
cad          2
appet        1
pe           1
ane          1
ckd_yes      0
dtype: int64

### Setting X and y, split to train and test

In [37]:
ckd_new = ckd[['age', 'bp', 'sg', 'al', 'su', 'bgr', 
               'bu', 'sc', 'sod', 'pot', 'hemo', 'pcv', 
               'ckd_yes', 'appet']]

ckd_new = ckd_new.dropna(how='any')
ckd_new.shape
ckd_new['appet'].value_counts()

good    190
poor     32
Name: appet, dtype: int64

In [38]:
ckd_new['appet'] = ckd_new['appet'].map(lambda x: 1 if x == 'good' else 0)
ckd_new = ckd_new.rename(
    columns={'appet': 'appet_good'}
)
ckd_new['appet_good'].value_counts()

1    190
0     32
Name: appet_good, dtype: int64

In [39]:
X = ckd_new.drop(columns='ckd_yes')
y = ckd_new['ckd_yes']

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state=42,
                                                    stratify=y
                                                   )

ss = StandardScaler()
X_train_sc = ss.fit_transform(X_train)
X_test_sc = ss.transform(X_test)

In [40]:
logreg = LogisticRegression()
logreg.fit(X_train_sc, y_train)

LogisticRegression()

---

## Step 5: Evaluate the model.

### 13. Based on your logistic regression model constructed in problem 12, interpret the coefficient of one of your quantitative features.

**Answer:** One increase in age makes you 1.36x more likely to have CKD.

In [41]:
pd.DataFrame(logreg.coef_[0], index=X.columns).T

,age,bp,sg,al,su,bgr,bu,sc,sod,pot,hemo,pcv,appet_good
0,0.310596,0.275792,-1.375012,1.403847,0.684432,0.358842,0.082236,0.963572,-0.231413,-0.177925,-1.694824,-1.497499,-0.234854


In [43]:
print(np.exp(0.310596))

1.3642379577035235


### 14. Based on your logistic regression model constructed in problem 12, interpret the coefficient of one of your categorical/dummy features.

**Answer:** If a person's appetite is good, they are 0.79x as likely to have CKD.

In [44]:
print(np.exp(-0.234854))

0.7906862813990956


### 15. Despite being a relatively simple model, logistic regression is very widely used in the real world. Why do you think that's the case? Name at least two advantages to using logistic regression as a modeling technique.

Answer:

1. Many things are 1 and 0. E.g. Buy or not buy.

2. Ease of interpretation, how X affects Y.

### 16. Does it make sense to generate a confusion matrix on our training data or our test data? Why? Generate it on the proper data.

> Hint: Once you've generated your predicted $y$ values and you have your observed $y$ values, then it will be easy to [generate a confusion matrix using sklearn](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html).

In [45]:
y_preds = logreg.predict(X_test_sc)


In [46]:
tn, fp, fn, tp = confusion_matrix(y_train, logreg.predict(X_train_sc)).ravel()
tn, fp, fn, tp

# Obviously this will be all

(94, 0, 0, 72)

In [47]:
tn, fp, fn, tp = confusion_matrix(y_test, y_preds).ravel()
confusion_matrix(y_test, y_preds)
tn, fp, fn, tp

(32, 0, 1, 23)

In [49]:
print(classification_report(y_test, y_preds))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98        32
           1       1.00      0.96      0.98        24

    accuracy                           0.98        56
   macro avg       0.98      0.98      0.98        56
weighted avg       0.98      0.98      0.98        56



### 17. In this hospital case, we want to predict CKD. Do we want to optimize for sensitivity, specificity, or something else? Why? (If you don't think there's one clear answer, that's okay! There rarely is. Be sure to defend your conclusion!)

Answer: You want to optimize on Sensitivity, because you want to minimize false negatives. It's better for false negatives to occur rather than false positives.

### 18 (BONUS). Write a function that will create an ROC curve for you, then plot the ROC curve.

Here's a strategy you might consider:
1. In order to even begin, you'll need some fit model. Use your logistic regression model from problem 12.
2. We want to look at all values of your "threshold" - that is, anything where .predict() gives you above your threshold falls in the "positive class," and anything that is below your threshold falls in the "negative class." Start the threshold at 0.
3. At this value of your threshold, calculate the sensitivity and specificity. Store these values.
4. Increment your threshold by some "step." Maybe set your step to be 0.01, or even smaller.
5. At this value of your threshold, calculate the sensitivity and specificity. Store these values.
6. Repeat steps 3 and 4 until you get to the threshold of 1.
7. Plot the values of sensitivity and 1 - specificity.

### 19. Suppose you're speaking with the biostatistics lead at Mayo Clinic, who asks you "Why are unbalanced classes generally a problem? Are they a problem in this particular CKD analysis?" How would you respond?

**Answer:** Unbalanced classes make it difficult for the model to learn the smaller group. Hence, we need to stratify during train_test_split. With a 57/44 no/yes split, there should not be a problem. Also, since we stratified during splitting, there is no problem.

In [56]:
print(f'Original data proportion: \n{ckd_new["ckd_yes"].value_counts(normalize=True)}\n')
print(f'Training data proportion: \n{y_train.value_counts(normalize=True)}\n')
print(f'Testing data proportion:  \n{y_test.value_counts(normalize=True)}')

Original data proportion: 
0    0.567568
1    0.432432
Name: ckd_yes, dtype: float64

Training data proportion: 
0    0.566265
1    0.433735
Name: ckd_yes, dtype: float64

Testing data proportion:  
0    0.571429
1    0.428571
Name: ckd_yes, dtype: float64


### 20. Suppose you're speaking with a doctor at Mayo Clinic who, despite being very smart, doesn't know much about data science or statistics. How would you explain why unbalanced classes are generally a problem to this doctor?

**Answer:** For instance, we want to create a model on whether or not someone will develop Multiple Sclerosis (35.9 per 100,000 population). Most people will develop it, and a few will. We want to split the data to 'train' the models, and then 'test' the models out. If, in the randomness of splitting the data, everyone in the training set does not have MS, then we won't know the difference between someone who *has* MS and someone who does not.

### 21. Let's create very unbalanced classes just for the sake of this example! Generate very unbalanced classes by [bootstrapping](http://stattrek.com/statistics/dictionary.aspx?definition=sampling_with_replacement) (a.k.a. random sampling with replacement) the majority class.

1. The majority class are those individuals with CKD.
2. Generate a random sample of size 200,000 of individuals who have CKD **with replacement**. (Consider setting a random seed for this part!)
3. Create a new dataframe with the original data plus this random sample of data.
4. Now we should have a dataset with around 200,000 observations, of which only about 0.00075% are non-CKD individuals.

In [57]:
ckd_unbalanced = ckd_new[ckd_new['ckd_yes'] == 1].sample(200_000,
                                                  replace = True,     
                                                  random_state = 42)  

In [82]:
ckd_bad = pd.concat([ckd_new, ckd_unbalanced], axis=0)
ckd_bad.reset_index(drop=True, inplace=True)
ckd_bad['ckd_yes'].value_counts(normalize=True)

1    0.999371
0    0.000629
Name: ckd_yes, dtype: float64

### 22. Build a logistic regression model on the unbalanced class data and evaluate its performance using whatever method(s) you see fit. How would you describe the impact of unbalanced classes on logistic regression as a classifier?
> Be sure to look at how well it performs on non-CKD data.

In [95]:
X_2 = ckd_bad.drop(columns='ckd_yes')
y_2 = ckd_bad['ckd_yes']

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, y_2,
                                                           )

ss = StandardScaler()
X_train_2_sc = ss.fit_transform(X_train_2)
X_test_2_sc = ss.transform(X_test_2)

logreg = LogisticRegression()
logreg.fit(X_train_2_sc, y_train_2)

print(logreg.score(X_train_2_sc, y_train_2))
print(logreg.score(X_test_2_sc, y_test_2))

tn, fp, fn, tp = confusion_matrix(y_train, logreg.predict(X_train_sc)).ravel()

print(classification_report(y_test_2, logreg.predict(X_test_2_sc)))

0.9999733628118216
0.9999001118747003
              precision    recall  f1-score   support

           0       1.00      0.86      0.92        35
           1       1.00      1.00      1.00     50021

    accuracy                           1.00     50056
   macro avg       1.00      0.93      0.96     50056
weighted avg       1.00      1.00      1.00     50056



In [97]:
# Sensitivity: TP / (TP + FN)
tp / (tp + fn)

# Specificity: TN / (TN + FP)
tn / (tn + fp)

0.0

---

## Step 6: Answer the problem.

At this step, you would generally answer the problem! In this situation, you would likely present your model to doctors or administrators at the hospital and show how your model results in reduced false positives/false negatives. Next steps would be to find a way to roll this model and its conclusions out across the hospital so that the outcomes of patients with CKD (and without CKD!) can be improved!